In [1]:
from urllib.request import urlopen
import json
from IPython.core.display import display, HTML

import pandas as pd
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import plotly.io as pio
from plotly.subplots import make_subplots
from ipywidgets import HBox,VBox
import numpy as np
import plotly.graph_objs as go
import plotly.express as px

#     for i in points.point_inds:
#         c[i] = '#bae2be'
#         s[i] = 20
        
#         with f.batch_update():
#             scatter.marker.color = c
#             scatter.marker.size = s

# counties=[]    
# with open('Countries.json','r') as response:
#     counties = json.load(response)
    
#df = px.data.gapminder()
#print(df)
raw_data = pd.read_csv("time_series_covid_19_confirmed.csv")


#exit(1)
# for i,data in enumerate(raw_data['iso3']):
# 	if data=='USA':
# 		raw_data.drop(i,axis=0,inplace=True)

#print(raw_data['Province_State'])

#print(df)

#print(raw_data)
#print()


var="5/31/20"

f=go.FigureWidget(px.scatter_geo(raw_data,scope="world",locationmode = 'country names',locations="COUNTRY",
                    color=var,color_continuous_scale='blues_r',
                 hover_name="COUNTRY"))


p=[]
flag=0
def controller(points):
    global p,fk,f1
    country=[]
    for i in points:
        count=raw_data.loc[[i]]['COUNTRY']
        country.append(count.values[0])
    #print(country)    
    country_data=call_me_for_more(country,1)
    #print(country_data)
    fig = px.line(country_data, x="dates", y="cases", color='country')
    fig.update_xaxes(rangeslider_visible=True)
    fid=go.FigureWidget(fig)
    f1=VBox([fk,fid])
    clear_output(wait=True)
    display(HTML(r"""<style id=hide>div.input{display:none;}</style><button type="button"onclick="var myStyle = document.getElementById('hide').sheet;myStyle.insertRule('div.input{display:inherit !important;}', 0);">Show Code</button>"""))
    display(f1)

        
flag=0
def update_point(trace, points, selector):
    global flag,fk
    #print("Updating plot")
    #c = list(scatter.marker.name)
     
    for i in points.point_inds:
        #print(c[i])
        #print(i)
        if (selector.ctrl==True):
            #print("called")
            p.append(i)
            flag+=1
            return
        if flag>=1 and selector.ctrl==False:
            controller(p)
            flag=0
            return  
        
                
        
        country=raw_data.loc[[i]]['COUNTRY']
        country=country.values[0]
        country_data=call_me_for_more(country)
        
        fig1 = px.line(country_data, x="dates", y="cases", color='country')
        
        
        #fig.show()
        
        fig = go.Figure(
                data=[go.Scatter(x=[0], y=[0],mode="markers")],
                layout=go.Layout(
                yaxis=dict(range=[0, country_data['cases'].values[-1]], autorange=False),
                xaxis=dict(range=[0, 132], autorange=False),
                xaxis_title="Day",
                yaxis_title="Cases",
                title=country_data['country'].values[-1],
                updatemenus=[dict(
                    type="buttons",
                    buttons=[dict(label="Play",
                                  method="animate",
                                  args=[None, {"frame": {"duration": 20, "redraw": False},}])])]
            ),
             frames=[go.Frame(
                data=[go.Scatter(
                    x=np.arange(i),
                    y=country_data["cases"][0:i],
                    mode="lines",
                    marker=dict(color="red", size=10))])

                for i in range(len(country_data["cases"].values))]
        )
        
        fig.update_layout(transition_duration=0)
        
        fig_=fig
        
        fig=go.FigureWidget(fig1)
        
        f=VBox([fk,fig])
        clear_output(wait=True)
        
        display(HTML(r"""<style id=hide>div.input{display:none;}</style><button type="button"onclick="var myStyle = document.getElementById('hide').sheet;myStyle.insertRule('div.input{display:inherit !important;}', 0);">Show Code</button>"""))
        fig_.show()
        display(f)
        
        
        #print(trace)
        #print(selector)
        
def call_me_for_more(country,label=0):
    df_store=pd.DataFrame()
    count=0
    if label==0:
        for i,data in enumerate(raw_data['COUNTRY']):
            if data==country:
                df_new = raw_data.iloc[i][4:]
                df_new=df_new.T
                df_new = df_new.reset_index()
                #df_new = df_new.set_index(['date', 'cases'])
                df_new=df_new.rename(columns={df_new.keys()[0]: 'dates',df_new.keys()[1]:'cases'})
                
                df_new['country']=data
#                 x=[]
#                 y=[]
#                 for case in df_new['cases']:
#                     x.append(np.random.randint(1,100,case))
#                     y.append(np.random.randint(1,100,case))
                
#                 df_new['x_rand']=x
#                 df_new['y_rand']=y
                
                #print(df_new.keys())
                return df_new
    else:
        for i,data in enumerate(raw_data['COUNTRY']):
            if data in country:
                #print(data)
                df_new = raw_data.iloc[i][4:]
                df_new=df_new.T
                df_new = df_new.reset_index()
                #df_new = df_new.set_index(['date', 'cases'])
                df_new=df_new.rename(columns={df_new.keys()[0]: 'dates',df_new.keys()[1]:'cases'})
                df_new['country']=data
                if count==0:
                    df_store=(df_new)
                    #print("once")
                else:
                    df_store = pd.concat([df_store, df_new])
                    #print(df_new)
                    #print("many")
                count+=1
                #print(df_new.keys())
        #print(df_store)
        return df_store

            
        
        
        
def selection_fn(trace,points,selector):
    country=[]
    global fk,f1
    for i in points.point_inds:
        #print(c[i])
        #print(i)
        
        count=raw_data.loc[[i]]['COUNTRY']
        country.append(count.values[0])
    #print(country)    
    country_data=call_me_for_more(country,1)
    #print(country_data)
    #fig = go.FigureWidget(px.line(country_data, x="dates", y="cases", color='country'))
    clear_output(wait=True)
    fig=go.FigureWidget(px.line(country_data, x="dates", y="cases", color='country'))
    f1=VBox([fk,fig])
    display(HTML(r"""<style id=hide>div.input{display:none;}</style><button type="button"onclick="var myStyle = document.getElementById('hide').sheet;myStyle.insertRule('div.input{display:inherit !important;}', 0);">Show Code</button>"""))
    display(f1)
    
    
    
    #fig.update_xaxes(rangeslider_visible=True)
    #clear_output(wait=True)
#     scatter=f.data[0]
#     scatter.on_click(update_point)
#     scatter.on_selection(selection_fn)
    
    #box_me=HBox([f,fig])
    #return "edfgbnwesdffg"
    
scatter=f.data[0]
scatter.on_click(update_point)

scatter.on_selection(selection_fn)

f.update_traces(textposition='top center')

f.update_layout(
    height=500,
    title_text='COVID 19 WORLD PLOT'
)

fk=f
f=VBox([f])
display(HTML(r"""<style id=hide>div.input{display:none;}</style><button type="button"onclick="var myStyle = document.getElementById('hide').sheet;myStyle.insertRule('div.input{display:inherit !important;}', 0);">Show Code</button>"""))
display(f)

# with open('p_graph.html', 'a') as f1:
#     f1.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
#     f1.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
#     f1.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))


    'data': [{'geo': 'geo',
              'hovertemplate': ('<b>%{hovertext}</b>…